In [2]:
# CARREGAR BANCO DE DADOS
# AJUSTAR COLUNAS

import pandas as pd

caminho_dados = 'dados/acidentes.csv'
separador_original = ';'
codificacao = 'utf-8'

caminho_dados_virgula = 'dados/acidentes-virgula.csv'
separador_virgula = ','

try:
    # 1. Carregar o CSV original usando o separador correto (;)
    df = pd.read_csv(caminho_dados, sep=separador_original, encoding=codificacao)

    print("CSV original lido com sucesso! Primeiras 5 linhas:")
    print(df.head())
    print("\nInformações do DataFrame original:")
    df.info()

    # 2. Salvar o DataFrame em um novo arquivo CSV usando a vírgula (,) como separador
    # index=False evita que o Pandas grave o índice do DataFrame como uma nova coluna no CSV
    df.to_csv(caminho_dados_virgula, sep=separador_virgula, encoding=codificacao, index=False)

    print(f"\nArquivo ajustado salvo como '{caminho_dados_virgula}' com separador '{separador_virgula}'.")


except FileNotFoundError:
    print(f"Erro: Arquivo '{caminho_dados}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

CSV original lido com sucesso! Primeiras 5 linhas:
         data_extracao  predial1  queda_arr                 data  feridos  \
0  2025-06-01 01:33:13       0.0        0.0  2020-10-17 00:00:00        1   
1  2025-06-01 01:33:13     598.0        0.0  2020-01-01 00:00:00        1   
2  2025-06-01 01:33:13    1271.0        0.0  2020-01-01 00:00:00        1   
3  2025-06-01 01:33:13    1901.0        0.0  2020-01-02 00:00:00        2   
4  2025-06-01 01:33:13    3302.0        0.0  2020-01-02 00:00:00        1   

   feridos_gr  mortes  morte_post  fatais  auto  ...  longitude   latitude  \
0           0       0           0       0     3  ...   0.000000   0.000000   
1           0       0           0       0     0  ...        NaN        NaN   
2           1       0           0       0     1  ...        NaN        NaN   
3           0       0           0       0     0  ...        NaN        NaN   
4           0       0           0       0     1  ... -51.211535 -30.081535   

                 

In [ ]:
# LIMPA NOMES COLUNAS

df = pd.read_csv('dados/acidentes-virgula.csv')

print("\n--- Nomes de todas as colunas no seu DataFrame: ---")
print(df.columns.tolist()) # Isso vai te dar a lista exata dos nomes das colunas

# Cria variável para armazenar coluna
coluna_data = 'data'

# Remove espaços em branco no início/fim dos nomes das colunas (boa prática)
df.columns = df.columns.str.strip()

# Re-imprime para ver se o nome da coluna de data foi limpo
print("\n--- Nomes das colunas após remover espaços em branco: ---")
print(df.columns.tolist())


In [6]:
# TRANSFORMA TIPO DATA
# EXCLUI DATAS INVÁLIDAS

coluna_data = 'data'
coluna_id = 'idacidente'

if coluna_data in df.columns and coluna_id in df.columns:

    print(f"\nColunas '{coluna_data}' e '{coluna_id}' encontradas no DataFrame.")

    # Converte a Coluna de Data para Datetime
    print(f"Tipo de dado atual da coluna '{coluna_data}': {df[coluna_data].dtype}")
    df[coluna_data] = pd.to_datetime(df[coluna_data], errors='coerce')
    print(f"Tipo de dado após a conversão: {df[coluna_data].dtype}")

    # Identifica e Imprimi ID do Acidente de Datas Inválidas
    # Cria um DataFrame contendo apenas as linhas onde a coluna de data é NaT (inválida)
    datas_invalidas_df = df[df[coluna_data].isna()]
    datas_invalidas_count = len(datas_invalidas_df)

    if datas_invalidas_count > 0:
        print(f"\nATENÇÃO: {datas_invalidas_count} datas inválidas (NaT) foram geradas na coluna '{coluna_data}'.")
        print(f"IDs de acidente para as datas inválidas:")
        print(f"DF datas invalidas\n{datas_invalidas_df}")
        # Itera sobre o DataFrame de datas inválidas e imprime o ID do acidente para cada linha
        for idx, row in datas_invalidas_df.iterrows():
            print(f"- ID do Acidente: {row[coluna_id]}")
            # Remove linha com data inválida
            df = df.dropna(subset=[coluna_data])
            primiera_linha = len(df)
            rows_removed = primiera_linha - len(df)
        print(f"\n{rows_removed} linhas com datas inválidas foram removidas do DataFrame.")
    else:
        print(f"\nColuna '{coluna_data}' convertida para datetime sem NaTs.")

    


Colunas 'data' e 'idacidente' encontradas no DataFrame.
Tipo de dado atual da coluna 'data': datetime64[ns]
Tipo de dado após a conversão: datetime64[ns]

Coluna 'data' convertida para datetime sem NaTs.


In [ ]:
# Filtra o DataFrame por Ano
ano_inicio = 2020
ano_fim = 2024

# A filtragem agora usa o coluna_data
df_filtrado_por_ano = df[
    (df[coluna_data].dt.year >= ano_inicio) &
    (df[coluna_data].dt.year <= ano_fim)
]

print(f"\n--- DataFrame Filtrado por Ano ({ano_inicio} a {ano_fim}) ---")
print(f"Número de linhas no DataFrame original: {len(df)}")
print(f"Número de linhas no DataFrame filtrado: {len(df_filtrado_por_ano)}")

print("\n>>> Primeiras 5 linhas do DataFrame Filtrado:")
print(df_filtrado_por_ano.head())
print("\n>>> Contagem de registros por ano (filtrado):")
print(df_filtrado_por_ano[coluna_data].dt.year.value_counts().sort_index())



--- DataFrame Filtrado por Ano (2020 a 2024) ---
Número de linhas no DataFrame original: 69520
Número de linhas no DataFrame filtrado: 66120

>>> Primeiras 5 linhas do DataFrame Filtrado:
         data_extracao  predial1  queda_arr       data  feridos  feridos_gr  \
0  2025-06-01 01:33:13       0.0        0.0 2020-10-17        1           0   
1  2025-06-01 01:33:13     598.0        0.0 2020-01-01        1           0   
2  2025-06-01 01:33:13    1271.0        0.0 2020-01-01        1           1   
3  2025-06-01 01:33:13    1901.0        0.0 2020-01-02        2           0   
4  2025-06-01 01:33:13    3302.0        0.0 2020-01-02        1           0   

   mortes  morte_post  fatais  auto  ...  longitude   latitude  \
0       0           0       0     3  ...   0.000000   0.000000   
1       0           0       0     0  ...        NaN        NaN   
2       0           0       0     1  ...        NaN        NaN   
3       0           0       0     0  ...        NaN        NaN   
4     